In [ ]:
#Training sets
train_features_rgb_r = 
train_features_rgb_g= np.load('../data/features/histo_rgb_g_train.npy')
train_features_rgb_b = np.load('../data/features/histo_rgb_b_train.npy')
print('shape train rgb r', train_features_rgb_r.shape)
print('shape train rgb g', train_features_rgb_g.shape)
print('shape train rgb b', train_features_rgb_b.shape)

train_features_rgb = np.concatenate((train_features_rgb_r, train_features_rgb_g, train_features_rgb_b), axis=1)
print('Shape train rgb', train_features_rgb.shape)

#Testing sets
test_features_rgb_r = np.load('../data/features/histo_rgb_r_test.npy')
test_features_rgb_g = np.load('../data/features/histo_rgb_g_test.npy')
test_features_rgb_b = np.load('../data/features/histo_rgb_b_test.npy')
print('shape test rgb r', test_features_rgb_r.shape)
print('shape test rgb g', test_features_rgb_g.shape)
print('shape test rgb b', test_features_rgb_b.shape)

test_features_rgb = np.concatenate((test_features_rgb_r, test_features_rgb_g, test_features_rgb_b), axis=1)
print('Shape test rgb', test_features_rgb.shape)

In [ ]:
# Training sets
train_features_lab_l = np.load('../data/features/histo_lab_l_train.npy')
train_features_lab_a= np.load('../data/features/histo_lab_a_train.npy')
train_features_lab_b = np.load('../data/features/histo_lab_b_train.npy')
print('shape train lab l', train_features_lab_l.shape)
print('shape train lab a', train_features_lab_a.shape)
print('shape train lab b', train_features_lab_b.shape)

train_features_lab = np.concatenate((train_features_lab_l, train_features_lab_a, train_features_lab_b), axis=1)

# Testing sets
test_features_lab_l = np.load('../data/features/histo_lab_l_test.npy')
test_features_lab_a = np.load('../data/features/histo_lab_a_test.npy')
test_features_lab_b = np.load('../data/features/histo_lab_b_test.npy')
print('shape test lab l', test_features_lab_l.shape)
print('shape test lab a', test_features_lab_a.shape)
print('shape test lab b', test_features_lab_b.shape)

test_features_lab = np.concatenate((test_features_lab_l, test_features_lab_a, test_features_lab_b), axis=1)

In [ ]:
k=3
predicted_genres = KNN(clean_movies, train_posters, train_features_resnet, train_genres, train_ids, test_posters, test_features_resnet, test_ids, -1, k)

In [ ]:
# POUR LA METHODE RESNET :

RESNET = False #encore à faire

# POUR LES HISTOGRAMMES :

# On renseigne d'abord si on veut l'histogramme pour la couleur considérée, puis le nombre de bins
# Attention, les histogrammes avec le nombre de bins correspondants doivent avoir été calculés

RGB_R = False
RGB_R_bins = 256

RGB_G = False
RGB_G_bins = 256 

RGB_B = False
RGB_B_bins = 256 

LAB_L = True
LAB_L_bins = 16

LAB_A = True
LAB_A_bins = 16

LAB_B = True
LAB_B_bins = 16